Author:
        
        KIM, JeongYoong, jeongyoong@ccnets.org
        
    COPYRIGHT (c) 2024. CCNets. All Rights reserved.

<p align="center">
  <img src="https://storage.googleapis.com/kaggle-datasets-images/4956778/8344638/a2a6aa289fce8461958dc287f1dab799/dataset-cover.jpg?t=2024-05-07-09-36-53" alt="IMG">
</p>

<h1 style = 'font-family: Times New Roman'> <b>|</b><i> 1. Content</i></h1>

<br>

> <h4 style = 'font-family: Times New Roman'>
This dataset explores how weather conditions impact renewable energy generation. <br><br>Spanning from January 1, 2017, to August 31, 2022, the dataset provides climate data such as temperature, pressure, wind speed, and sunlight duration at 15-minute intervals. <br><br>By incorporating variables like GHI and SunlightTime, it enables the prediction of solar energy production.


- DataSource: https://www.kaggle.com/datasets/pythonafroz/renewable-power-generation-and-weather-conditions/data
  
<h1 style = 'font-family: Times New Roman'> <b>|</b><i> 2. About Dataset</i></h1>

<br> 

<details>
    <summary>More Columns Info</summary>
    17 columns in the dataset, below is <u>description of main features</u>:

    *  (1) Time: The timestamp of the recorded data in the format of YYYY-MM-DD HH:MM:SS.
        
    *  (2) Energy delta[Wh]: The difference in energy consumption in Watt-hours (Wh) from the previous timestamp to the current timestamp.
        
    *  (3) GHI: Global Horizontal Irradiance in Watts per square meter (W/m²) measured by a pyranometer.
        
    *  (4) temp: The temperature in degrees Celsius (°C) measured at the same height as the pyranometer.

    *  (5) pressure: The atmospheric pressure in hectopascals (hPa) measured at the same height as the pyranometer.

    *  (6) humidity: The relative humidity in percentage (%) measured at the same height as the pyranometer.

    *  (7) wind_speed: The wind speed in meters per second (m/s) measured at the same height as the pyranometer.

    *  (8) rain_1h: The amount of precipitation in millimeters (mm) measured over the past hour.
    
    *  (9) snow_1h: The amount of snowfall in millimeters.

    *  (10) clouds_all: The cloud situation.
</details>    
    
<h1 style = 'font-family: Times New Roman'> <b>|</b><i> 3. Goal of the Notebook</i></h1>
    
> <h4 style = 'font-family: Times New Roman'>
Goal is to train and test model using GPT, Pytorch. <br><br>
    The Target Column used here is <b>Energy delta[Wh]</b>
</h4>

 

***

<a id="1"></a>
> <h1 style = 'font-family: Times New Roman'><b> <b style = 'color: #42c2f5'>1.</b> Import Necessary Libraries </b></h1>

In [1]:
import os
import sys
import warnings
warnings.filterwarnings("ignore")

path_append = "../"
sys.path.append(path_append)  # Go up one directory from where you are.

In [2]:
import pandas  as pd
df = pd.read_csv(path_append + '../data/Renewable Power Generation and weather Conditions/Renewable.csv')
df.head()

,Time,Energy delta[Wh],GHI,temp,pressure,humidity,wind_speed,rain_1h,snow_1h,clouds_all,isSun,sunlightTime,dayLength,SunlightTime/daylength,weather_type,hour,month
0,2017-01-01 00:00:00,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0,4,0,1
1,2017-01-01 00:15:00,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0,4,0,1
2,2017-01-01 00:30:00,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0,4,0,1
3,2017-01-01 00:45:00,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0,4,0,1
4,2017-01-01 01:00:00,0,0.0,1.7,1020,100,5.2,0.0,0.0,100,0,0,450,0.0,4,1,1


<a id="2"></a>
> <h1 style = 'font-family: Times New Roman'><b> <b style = 'color: #4290f5'>2.</b> Modeling: Preprocess </b></h1>

In [3]:
df['Time'] = pd.to_datetime(df['Time'], format='%Y-%m-%d %H:%M:%S')

# Extract time in seconds from the datetime object
df['SecondsOfDay'] = df['Time'].dt.hour * 3600 + df['Time'].dt.minute * 60 + df['Time'].dt.second

# Optionally drop the original Time column
df.drop(['Time'], axis=1, inplace=True)

# Display the first few rows to verify the changes
df.head()

,Energy delta[Wh],GHI,temp,pressure,humidity,wind_speed,rain_1h,snow_1h,clouds_all,isSun,sunlightTime,dayLength,SunlightTime/daylength,weather_type,hour,month,SecondsOfDay
0,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0,4,0,1,0
1,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0,4,0,1,900
2,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0,4,0,1,1800
3,0,0.0,1.6,1021,100,4.9,0.0,0.0,100,0,0,450,0.0,4,0,1,2700
4,0,0.0,1.7,1020,100,5.2,0.0,0.0,100,0,0,450,0.0,4,1,1,3600


In [4]:
from tools.preprocessing.data_frame import process_dataframe, preprocess_cyclical_columns
target_columns = ['Energy delta[Wh]']
one_hot_columns = ['weather_type']
df = preprocess_cyclical_columns(df, ['month'])
df, description = process_dataframe(df, target_columns, one_hot_columns = one_hot_columns, exclude_scale_columns = target_columns) 

description

{'num_features': 21,
 'num_classes': 1,
 'encoded_columns': Index([], dtype='object'),
 'target_encoded_columns': Index([], dtype='object'),
 'scalers': {'GHI': 'robust',
  'SecondsOfDay': 'minmax',
  'SunlightTime/daylength': 'minmax',
  'clouds_all': 'standard',
  'dayLength': 'minmax',
  'hour': 'minmax',
  'humidity': 'robust',
  'isSun': 'minmax',
  'month_cos': 'minmax',
  'month_sin': 'minmax',
  'pressure': 'robust',
  'rain_1h': 'robust',
  'snow_1h': 'robust',
  'sunlightTime': 'robust',
  'temp': 'robust',
  'wind_speed': 'robust'},
 'target_scalers': {}}

In [5]:
df.min(), df.max() 

(GHI                      -0.034188
 temp                     -2.140496
 pressure                 -3.545455
 humidity                 -2.818182
 wind_speed               -1.541667
 rain_1h                   0.000000
 snow_1h                   0.000000
 clouds_all               -1.801176
 isSun                    -1.000000
 sunlightTime             -0.076923
 dayLength                -1.000000
 SunlightTime/daylength   -1.000000
 hour                     -1.000000
 SecondsOfDay             -1.000000
 month_sin                -1.000000
 month_cos                -1.000000
 weather_type_1            0.000000
 weather_type_2            0.000000
 weather_type_3            0.000000
 weather_type_4            0.000000
 weather_type_5            0.000000
 Energy delta[Wh]          0.000000
 dtype: float64,
 GHI                          4.863248
 temp                         2.190083
 pressure                     2.818182
 humidity                     0.727273
 wind_speed                   4.416

In [6]:
import torch
from sklearn.model_selection import train_test_split
from tools.preprocessing.template_dataset import TemplateDataset

max_seq_len = 20
min_seq_len = 10
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)
# predict the next value in the sequence
train_df_x = train_df.iloc[:, :-1] # all columns except the last one
train_df_y = train_df.iloc[:, -1:] # only the last column

test_df_x = test_df.iloc[:, :-1] # all columns except the last one
test_df_y = test_df.iloc[:, -1:] # only the last column

kwargs = {'min_seq_len': min_seq_len, 'max_seq_len': max_seq_len}

print('train df shape: ', train_df.shape)
print('test df shape: ', test_df.shape)
trainset = TemplateDataset(train_df_x, train_df_y, **kwargs)
testset = TemplateDataset(test_df_x, test_df_y, **kwargs)

train df shape:  (137743, 22)
test df shape:  (59033, 22)


<a id="3"></a>
> <h1 style = 'font-family: Times New Roman'><b> <b style = 'color: #427bf5'>3.</b> Modeling</b></h1>

In [7]:
from tools.setting.data_config import DataConfig
from tools.setting.ml_params import MLParameters
from trainer_hub import TrainerHub

num_features = description['num_features']
num_classes = description['num_classes']
data_config = DataConfig(dataset_name = 'renewable-power-gen-prediction', task_type='regression', obs_shape=[num_features], label_size=num_classes)

#  Set training configuration from the AlgorithmConfig class, returning them as a Namespace object.
ml_params = MLParameters(ccnet_network = 'gpt', encoder_network = 'none')
ml_params.algorithm.error_function = 'mae'

# Set the device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

# Initialize the TrainerHub class with the training configuration, data configuration, device, and use_print and use_wandb flags
trainer_hub = TrainerHub(ml_params, data_config, device, use_print=False, use_wandb=True) 

In [8]:
trainer_hub.train(trainset, testset)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: junhopark (ccnets). Use `wandb login --relogin` to force relogin


wandb: Adding directory to artifact (.\..\saved\renewable-power-gen-prediction\causal-learning)... Done. 0.0s


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Iterations:   0%|          | 0/2151 [00:00<?, ?it/s]

--------------------Test Metrics------------------------
mse: 1210841.0000
mae: 502.8843
r2: -0.2550

--------------------Test Metrics------------------------
mse: 1191662.0000
mae: 523.5331
r2: -0.2891

--------------------Test Metrics------------------------
mse: 1643275.1250
mae: 620.7944
r2: -0.2960

--------------------Test Metrics------------------------
mse: 1844557.6250
mae: 697.9760
r2: -0.3460

--------------------Test Metrics------------------------
mse: 1062813.0000
mae: 500.6917
r2: -0.2919

--------------------Test Metrics------------------------
mse: 1409263.3750
mae: 573.2842
r2: -0.2887

--------------------Test Metrics------------------------
mse: 1348334.2500
mae: 541.8038
r2: -0.2641

--------------------Test Metrics------------------------
mse: 1746167.3750
mae: 647.3327
r2: -0.3012

--------------------Test Metrics------------------------
mse: 1123616.5000
mae: 507.7791
r2: -0.2826

--------------------Test Metrics------------------------
mse: 1674191.0000
mae: 68

...

--------------------Test Metrics------------------------
mse: 1421355.2500
mae: 566.4999
r2: -0.0645

--------------------Test Metrics------------------------
mse: 1134306.8750
mae: 500.0374
r2: -0.0419

--------------------Test Metrics------------------------
mse: 1429852.2500
mae: 578.5916
r2: -0.0732

--------------------Test Metrics------------------------
mse: 1012114.7500
mae: 447.9540
r2: 0.0009

--------------------Test Metrics------------------------
mse: 1030304.5625
mae: 439.8305
r2: 0.0065

--------------------Test Metrics------------------------
mse: 783306.0625
mae: 384.1007
r2: 0.0244

--------------------Test Metrics------------------------
mse: 1341679.6250
mae: 548.5828
r2: -0.0554

--------------------Test Metrics------------------------
mse: 1147471.5000
mae: 489.7135
r2: -0.0236

--------------------Test Metrics------------------------
mse: 949323.5000
mae: 448.0014
r2: -0.0150

--------------------Test Metrics------------------------
mse: 1117187.3750
mae: 475.178

Iterations:   0%|          | 0/2151 [00:00<?, ?it/s]

--------------------Test Metrics------------------------
mse: 923605.6250
mae: 433.3554
r2: 0.0022

--------------------Test Metrics------------------------
mse: 1192700.2500
mae: 478.9457
r2: -0.0197

--------------------Test Metrics------------------------
mse: 1270002.8750
mae: 551.6343
r2: -0.0604

--------------------Test Metrics------------------------
mse: 1133169.1250
mae: 484.5345
r2: -0.0189

--------------------Test Metrics------------------------
mse: 1240492.8750
mae: 502.0727
r2: -0.0281

--------------------Test Metrics------------------------
mse: 1141414.0000
mae: 497.6818
r2: -0.0332

--------------------Test Metrics------------------------
mse: 863559.3125
mae: 419.1416
r2: 0.0006

--------------------Test Metrics------------------------
mse: 1147380.8750
mae: 482.7271
r2: -0.0264

--------------------Test Metrics------------------------
mse: 1289512.6250
mae: 524.7560
r2: -0.0415

--------------------Test Metrics------------------------
mse: 1252646.2500
mae: 529.16

Iterations:   0%|          | 0/2151 [00:00<?, ?it/s]

--------------------Test Metrics------------------------
mse: 1073151.7500
mae: 451.4717
r2: -0.0051

--------------------Test Metrics------------------------
mse: 1239322.5000
mae: 528.6456
r2: -0.0492

--------------------Test Metrics------------------------
mse: 889248.0000
mae: 405.0203
r2: 0.0200

--------------------Test Metrics------------------------
mse: 1056218.8750
mae: 464.8928
r2: -0.0078

--------------------Test Metrics------------------------
mse: 981242.6250
mae: 454.3062
r2: -0.0121

--------------------Test Metrics------------------------
mse: 850097.1875
mae: 386.7253
r2: 0.0296

--------------------Test Metrics------------------------
mse: 1194246.6250
mae: 462.3699
r2: -0.0039

--------------------Test Metrics------------------------
mse: 1164711.7500
mae: 492.6817
r2: -0.0320

--------------------Test Metrics------------------------
mse: 826334.1250
mae: 381.1676
r2: 0.0360



In [ ]:
trainer_hub.test(testset)